Team Name: 404NA <br>
Anish M Rao 01FB16ECS062 <br>
Malaika Vijay 01FB16ECS189 <br>
Manasa Jagadeesh 01FB16ECS471

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.metrics import mean_squared_error,classification_report,accuracy_score
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
print(os.listdir('../input'))
data = pd.read_csv('../input/Absenteeism_at_work.csv', delimiter=',')
X_orig=data.drop('Absenteeism time in hours',axis=1)
y=data['Absenteeism time in hours']
scaler=MinMaxScaler()
X = scaler.fit_transform(X_orig)
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=13,test_size=0.2)
#Note:
# Not scaling the target values because it will influence the mean squared error and other parameters.

DECISION TREE CLASSIFIER:
This piece of code tests values of max_depth and min_samples_split to determine the optimal value to be used in the Decision Tree Classifier

In [ ]:
# acc=[]
# for i in range(2,30):
#     for j in range(2,20):
#         classifier=DecisionTreeClassifier(criterion='entropy',max_depth=i,min_samples_split=j,random_state=13)
#         classifier.fit(X_train,y_train)
#         preds=classifier.predict(X_test)
#         acc.append((i,j,accuracy_score(y_test,preds)))
# acc[-1]

In [ ]:
classifier=DecisionTreeClassifier(criterion ='entropy', random_state = 13, max_depth=5, min_samples_split=2)
classifier.fit(X_train,y_train)
pred=classifier.predict(X_test)
print(classification_report(y_test,pred))
print("Accuracy: ",accuracy_score(y_test,pred))

**K-Nearest-Neighbors Regression**
Find attributes with high correlation first, to prune some of them

In [ ]:
def get_redundant_pairs(df):
    '''Get diagonal and lower triangular pairs of correlation matrix'''
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations(df, n=5):
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]

print("Top Absolute Correlations")
print(get_top_abs_correlations(data, 15))

In [ ]:
#Selected attributes for regression after pruning:
x_columns = ['Reason for absence',  'Month of absence','Day of the week',
             'Transportation expense', 'Distance from Residence to Work', 'Age', 'Work load Average/day ', 
             'Hit target', 'Disciplinary failure', 'Education']

X_reduced = X_orig[x_columns]
X_reduced = scaler.fit_transform(X_reduced)
X_train,X_test,y_train,y_test=train_test_split(X_reduced,y,random_state=13,test_size=0.2)

rmses = []
all_predictions = dict()
ks = range(1,19*2+1)

#picking k based on lowest rmse value
for k in ks:
    knn = KNeighborsRegressor(n_neighbors=k)
    knn.fit(X_train, y_train)
    predictions = knn.predict(X_test)
    all_predictions[k] = predictions
    rmses.append(np.sqrt(mean_squared_error(y_test, predictions)))

plt.plot(ks, rmses)
plt.show()
optimum_k = ks[rmses.index(min(rmses))]
print('K :', optimum_k, '\nrmse:', min(rmses))
final_preds = all_predictions[optimum_k]
knn = KNeighborsRegressor(n_neighbors=optimum_k)
knn.fit(X_train, y_train)
predictions = knn.predict(X_test)

**Conclusion**
The models cannot be compared directly from the metrics we have calculated since one is a classification method and other is a regression one. However, based on the nature of the features and the target variable (being continuous), a regression method will fit this problem better. This reflects in the reasonable RMSE value of the KNN regression model.